## Imports

In [1]:
import numpy as np

## Simple HRTF distance simulation

In [12]:
# Inverse of the speed of sound in air
delta = 1/343.0
# Initial delay for HUTUBS HRIRs measured at 1.47m radius
measured_radius = 1.5
initial_delay = measured_radius * delta
# Sampling rate
sr = 44100

def distance_delay(sr, ir, distance):
    # Delays impulse response for the given distance
    delay = distance * delta
    delay_gap = delay - initial_delay
    delay_samples = int(abs(delay_gap) * sr)
    if delay_gap > 0:
        # Add zeros to the beginning of the impulse response
        ir = np.pad(ir, (delay_samples, 0), 'constant')
    elif delay_gap < 0 and delay_samples < len(ir):
        # Remove zeros from the beginning of the impulse response
        ir = ir[delay_samples:]
    return ir

def distance_gain(ir, distance):
    # Scales impulse response for the given distance
    g = measured_radius / distance
    return g * ir

def distance_ir(sr, ir, distance):
    # Delays and scales impulse response for the given distance
    ir = distance_delay(sr, ir, distance)
    ir = distance_gain(ir, distance)
    return ir